# Qcodes example with Keysight B2200 Series Femto Leakage Switch Matrix

## Instrument Short info

### Physical grouping
The Keysight B1500 Semiconductor Parameter Analyzer consists of a *Mainframe* and can be equipped with various instrument *Modules*. 10 *Slots* are available in which up to 10 Modules can be installed (some modules occupy two slots). Each *Module* can have one or two *channels*.

### Logical grouping
The measurements are typically done in one of the 20 measurement modes. The modes can be roughly subdivided into 
  - Spot measurements
  - Pulsed Spot measurement
  - Sweep Measurements
  - Search Measurements


## Qcodes driver info
As can be see already from the instrument short info, the instrument is very versatile, but also very complex. Hence the driver will eventually consist of two layers:
  - The Low Level driver allows one to utilize all functions of the driver by offering a thin wrapper around the FLEX command set that the B1500 understands. 
  - A higher level driver that provides a simpler interface to the more frequently used features.

### Low level driver
The Low Level driver (`MessageBuilder` class) provides a wrapper function for each FLEX command. The commands are not executed right away, but rather assemble a message string which later can be sent to the instrument. This is done to enable fast execution, but is also important if one wants to make use of the internal programming function of the B1500 (complex measurement routines can be stored into the internal memory of the B1500 and then be called later)

### High level driver
The high level driver is still under development.

In [1]:
import qcodes as qc

from qcodes.instrument_drivers.Keysight.keysightb1500 import KeysightB1500
from pyvisa.errors import VisaIOError

from IPython.display import display, Markdown

In [2]:
# Create a station to hold all the instruments

station = qc.Station()

# instantiate the Switch Matrix and add it to the station
try:
    b1500 = KeysightB1500('switch_matrix',
                        address='TCPIP::192.168.111.100::gpib0,22::INSTR')
    display(Markdown("**Note: using physical instrument.**"))
except (ValueError, VisaIOError):
    # Either there is no VISA lib installed or there was no real instrument found at the
    # specified address => use simulated instrument
    import qcodes.instrument_drivers.Keysight.keysightb1500 as module
    path_to_yaml = module.__file__.replace('__init__.py', 'keysight_b1500.yaml')

    b1500 = KeysightB1500('SPA',
                          address='GPIB::1::INSTR',
                          visalib=path_to_yaml + '@sim')
    display(Markdown("**Note: using simulated instrument.**"))

station.add_component(b1500)

**Note: using simulated instrument.**

'SPA'

## Low Level Driver


In [ ]:
Sub perform_meas(ByVal session As IMessage, ByVal t() As Integer) ’1
Dim i As Integer = 0
Dim j As Integer = 0
Dim nop1 As Integer = 1
Dim nop2 As Integer = 1
Dim value As String = "Vg (V), Cgb (pF), C-status, Ileak (pA), I-status, Time
(sec)"
Dim fname As String = "C:\Keysight\prog_ex\data21.txt" Dim title As String = "QSCV Measurement Result"
Dim msg As String = "No error."
Dim err As Integer = 0
Dim vg1 As Double = 3.2 ’12 Dim vg2 As Double = -7.2
Dim vstep As Double = 0.2
Dim gstep As Integer = Math.Round(Math.Abs(vg2 - vg1) / Math.Abs(vstep)) - 1
Dim cvoltage As Double = 0.25 Dim icomp As Double = 0.1
Dim swp As Integer = 1
Dim hold As Double = 5
Dim delay1 As Double = 0.0 Dim delay2 As Double = 0.0 Dim cinteg As Double = 0.1 Dim linteg As Double = 0.1 Dim range As Integer = -10 nop1 = gstep
Dim data(nop2-1, nop1-1) As String ’26 Dim rep As Integer
Dim mret As String
Dim sc(nop1) As Double
Dim md1(nop1) As Double
Dim st1(nop1) As String
Dim md2(nop1) As Double
Dim st2(nop1) As String
Dim tm(nop1) As Double

session.Timeout = 60000
session.WriteString("FMT 1,1" & vbLf) session.WriteString("TSC 1" & vbLf)
session.WriteString("MM 13," & t(1) &
session.WriteString("QSC 0" & vbLf)
session.WriteString("QSL 1,1" & vbLf)
session.WriteString("QSM 2,1" & vbLf)
session.WriteString("QSR " & range & vbLf)
session.WriteString("QST " & cinteg & "," & linteg & "," & hold & "," & delay1 &
"," & delay2 & vbLf)
session.WriteString("QSV " & t(1) & "," & swp & ",0," & vg1 & "," & vg2 & "," &
cvoltage & "," & gstep & "," & icomp & vbLf)
session.WriteString("ERR? 1" & vbLf) : err = session.ReadString(4 + 2) If err <> 0 Then session.WriteString("DZ" & vbLf) : GoTo Check_err
session.WriteString("QSZ 0" & vbLf) ’51
Dim rbx As Integer
rbx = MsgBox("Do you want to perform offset cancel?", vbYesNo, "") If rbx = vbYes Then
MsgBox("Open measurement terminal. Then click OK.", vbOKOnly, "") Console.WriteLine("Wait a minute . . ." & Chr(10)) session.WriteString("QSZ 2" & vbLf)
session.WriteString("*OPC?" & vbLf) : rep = session.ReadString(1 + 2) session.WriteString("ERR? 1" & vbLf) : err = session.ReadString(4 + 2) If err <> 0 Then session.WriteString("DZ" & vbLf) : GoTo Check_err mret = session.ReadString(16 + 2)
md1(0) = Val(Mid(mret, 4, 12))
Console.WriteLine("Offset data = " & md1(0) * 1000000000000.0 & "pF" & Chr(10)) MsgBox("Offset data = " & md1(0) * 1000000000000.0 & "pF", vbOKOnly, "") session.WriteString("QSZ 1" & vbLf)
End If ’67
                    
MsgBox("Connect DUT. Then click OK.", vbOKOnly, "") Console.WriteLine("Wait a minute . . ." & Chr(10)) session.WriteString("DV " & t(0) & ",0,0,0.1,0" & vbLf) session.WriteString("DV " & t(2) & ",0,0,0.1,0" & vbLf) session.WriteString("DV " & t(3) & ",0,0,0.1,0" & vbLf) session.WriteString("TSR" & vbLf) session.WriteString("XE" & vbLf)
’69
session.WriteString("*OPC?" & vbLf) : rep = session.ReadString(1 + 2) ’77 session.WriteString("ERR? 1" & vbLf) : err = session.ReadString(4 + 2)
If err <> 0 Then session.WriteString("DZ" & vbLf) : GoTo Check_err session.WriteString("NUB?" & vbLf) : rep = session.ReadString(3 + 2)
If rep <> nop1 * 4 Then session.WriteString("DZ" & vbLf) : GoTo Check_nop
mret = session.ReadString(16 * 4 * nop1 + 2) ’4*nop1 data + terminator ’83
For i = 0 To nop1 - 1
st1(i) = Mid(mret, i * 16 * 4 + 16 * 1 + 1, 3)
st2(i) = Mid(mret, i * 16 * 4 + 16 * 2 + 1, 3)
tm(i) = Val(Mid(mret, i * 16 * 4 + 4, 12))
md1(i) = Val(Mid(mret, i * 16 * 4 + 16 * 1 + 4, 12))
md2(i) = Val(Mid(mret, i * 16 * 4 + 16 * 2 + 4, 12))
sc(i) = Val(Mid(mret, i * 16 * 4 + 16 * 3 + 4, 12))
data(j, i) = Chr(13) & Chr(10) & sc(i) & ", " & md2(i) * 1000000000000.0 & ", "
& st2(i) & ", " & md1(i) * 1000000000000.0 & ", " & st1(i) & ", " & tm(i) Next i
session.WriteString("DZ" & vbLf) ’95 save_data(fname, title, value, data, nop1, nop2, session, t)
Exit Sub
Check_err: ’99 session.WriteString("EMG? " & err & vbLf) : msg = session.ReadString(256) MsgBox("Instrument error: " & err & Chr(10) & msg, vbOKOnly, "")
Exit Sub
Check_nop: ’104 MsgBox("No. of data: " & rep & " (not " & nop1 * 4 & ")", vbOKOnly, "")
End Sub

In [6]:
from qcodes.instrument_drivers.Keysight.keysightb1500.constants import ChNr,IMeasRange, VMeasRange, \
    IOutputRange, VOutputRange, FMT, MM, QSC, QSM, QSV, QSZ

In [ ]:
b1500.mb.fmt(format_id=FMT.Format.ASCII_12_DIGITS_WITH_HEADER_CRLF_EOI, mode=FMT.Mode.ONLY_MEASUREMENT_DATA)
    .mm()

## High Level Driver